In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="./data/test.json")

def chunk_examples(examples):
    chunks = []
    for sentence in examples["text"]:
        chunks += [sentence[i:i + 50] for i in range(0, len(sentence), 50)]
    return {"chunks": chunks}

chunked_dataset = dataset.map(chunk_examples, batched=True, remove_columns= ['text', 'id'])
chunked_dataset['train'][:10]
dataset
chunked_dataset

In [ ]:
from arguments_dataset2 import ArgumentsDataset
from transformers import AutoTokenizer
from datasets import load_dataset

from commonfn import align_labels, get_token_index

label_to_index = {
    # convert label to index
    'O': 0,
    'B-Subject': 1,
    'I-Subject': 2,
    'B-Equipment': 3,
    'I-Equipment': 4,
    'B-Date': 5,
    'I-Date': 6,
    'B-Location': 7,
    'I-Location': 8,
    'B-Area': 9,
    'I-Area': 10,
    'B-Content': 11,
    'I-Content': 12,
    'B-Militaryforce': 13,
    'I-Militaryforce': 14,
    'B-Object': 15,
    'I-Object': 16,
    'B-Materials': 17,
    'I-Materials': 18,
    'B-Result': 19,
    'I-Result': 20,
    'B-Quantity': 21,
    'I-Quantity': 22
}

label_names = {
    0: 'O',
    1: 'B-Subject',
    2: 'I-Subject',
    3: 'B-Equipment',
    4: 'I-Equipment',
    5: 'B-Date',
    6: 'I-Date',
    7: 'B-Location',
    8: 'I-Location',
    9: 'B-Area',
    10: 'I-Area',
    11: 'B-Content',
    12: 'I-Content',
    13: 'B-Militaryforce',
    14: 'I-Militaryforce',
    15: 'B-Object',
    16: 'I-Object',
    17: 'B-Materials',
    18: 'I-Materials',
    19: 'B-Result',
    20: 'I-Result',
    21: 'B-Quantity',
    22: 'I-Quantity'
}

tokenizer = AutoTokenizer.from_pretrained(
    '../models/distilbert-base-multilingual-cased-ner-hrl')

def __expand_and_align_labels__(examples):
        text = examples['text']
        labels = []
        data = {'input_ids': [], 'attention_mask': [], 'labels': []}
        for event in examples['event_list']:
            event_type = event['event_type']
            trigger_text = event['trigger']['text']
            trigger_offset = event['trigger']['offset']
            arguments = [
                (argument['role'], argument['text'], argument['offset'])
                for argument in event['arguments']
            ]
            labels.append((event_type, trigger_text,
                           trigger_offset, arguments))

        # 将文本转换为标记的ID
        input_tokens = tokenizer.tokenize(text)
        
        # 对齐标签到分词后的文本
        for event_type, trigger_text, trigger_offset, arguments in labels:
            # 对齐标签到分词后的文本
            prompt_position = get_token_index(
                text, input_tokens, trigger_offset[0])
            prompt_text = f'{trigger_text}，类型{event_type}，位置{prompt_position}'
            prompt_tokens = tokenizer.tokenize(prompt_text)

            # 将Prompt和输入文本的分词结果合并
            tokens = [tokenizer.cls_token] + prompt_tokens + \
                [tokenizer.sep_token] + input_tokens
            aligned_labels = align_labels(text, arguments, input_tokens)
            aligned_labels = ['O'] * \
                (len(tokens)-len(input_tokens))+aligned_labels
            label_ids = [label_to_index[label] for label in aligned_labels]
            inputs = tokenizer(tokens, is_split_into_words=True)
            data['input_ids'].append(inputs['input_ids'])
            data['attention_mask'].append(inputs['attention_mask'])
            data['labels'].append(label_ids)
        return {'data': data}
    
def to_flatten(examples):
        inputs={'input_ids':[],'attention_masks':[],'labels':[]}
        for ex_data in examples['data']:
            for i in range(len(ex_data['input_ids'])):
                inputs['input_ids'].append(ex_data['input_ids'][i])
                inputs['attention_masks'].append(ex_data['attention_mask'][i])
                inputs['labels'].append(ex_data['labels'][i])
        return inputs

def chunk_examples(examples):
    chunks = []
    #for sentence in examples["text"]:
    sentence=examples["text"]
    chunks += [sentence[i:i + 50] for i in range(0, len(sentence), 50)]
    return {"chunks": chunks}

def to_one(examples):
    chunks2=[]
    for chunks in examples["chunks"]:
        for sentence in chunks:
            chunks2 += [sentence]
    return {"chunks2": chunks2}
        

dataset = load_dataset("json", data_files= './data/FNDEE_train1.json')
processed_dataset = dataset.map(
        __expand_and_align_labels__, batched=False, remove_columns=["text", "event_list", "coref_arguments", "id"])


processed_dataset2=processed_dataset.map(to_flatten, batched=True, remove_columns=['data'])
processed_dataset2

In [ ]:
from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    AutoConfig,
)
from torch.utils.data import DataLoader
from event_dataset2 import EventDataset
tokenizer = AutoTokenizer.from_pretrained('../models/distilbert-base-multilingual-cased-ner-hrl')
event_dataset = EventDataset('./data/FNDEE_train1.json', './data/FNDEE_valid.json', tokenizer)
len(event_dataset['train'])

train_dataloader = DataLoader(
    event_dataset["train"],
    shuffle=True,
    batch_size=4
)

len(train_dataloader)

In [27]:
import torch
batch=[{'a':[1,2,3],'b':[4,5,6]},{'a':[7,8,9],'b':[10,11,12]}]
a_values = [torch.tensor(item['a']) for item in batch]
b_values = [torch.tensor(item['b']) for item in batch]
print(a_values,b_values)

[tensor([1, 2, 3]), tensor([7, 8, 9])] [tensor([4, 5, 6]), tensor([10, 11, 12])]
